In [59]:
import sys
sys.path.append('../../src')

In [60]:
import pandas
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.model.formulate import formulate, Constraints, Objective, ModelClass
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from itertools import product

from energiapy.components.result import Result
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.components.location import Location
from energiapy.plot import plot_results, plot_scenario
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.components.transport import Transport
from energiapy.components.scenario import Scenario
from energiapy.components.network import Network
from energiapy.components.location import Location
from energiapy.components.material import Material

from energiapy.components.process import Process, VaryingProcess
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.temporal_scale import TemporalScale
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
import matplotlib.pyplot as plt

from energiapy.model.weights import EmissionWeights


\begin{align}
\min_x \quad \theta^TH^Tx& + c^Tx\\
\text{s.t.} \quad Ax &\leq b + F\theta\\
A_{eq}x &= b_{eq}\\
A_\theta \theta &\leq b_\theta\\
x &\in R^n\\
\end{align}


In [61]:
scales = TemporalScale(discretization_list=[1])

In [62]:
bigM = 10**15
smallM = 0.0000000001

Declare Resources

In [63]:
Solar = Resource(name='Solar', cons_max=bigM, basis='MW', label='Solar Power', price= 0.01)

Wind = Resource(name='Wind', cons_max=bigM, basis='MW', label='Wind Power', price = 0.01)

Power = Resource(name='Power', basis='MW', demand=True,
                 label='Power generated')#, varying=[VaryingResource.DETERMINISTIC_DEMAND])

LiIPower = Resource(name='LiIPower', basis='MW',
                     store_max=bigM, label='Power stored')

GridPower = Resource(name='GridPower', basis='MW',
                     cons_max=bigM, label='Power bought')

H2O = Resource(name='H2O', cons_max=10**10,
               price=0.001, basis='kg', label='Water', block='Resource')

Declare Materials

In [64]:

LithiumCarbonateR = Material(name='LithiumCarbonateR', gwp=10602.89995, odp=0.0005311, acid=83.2005862, eutt=185.9710578, eutf=6.0273706, eutm=42.7091789, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Rock-based Lithium',
                citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  
LithiumCarbonateB = Material(name='LithiumCarbonateB', gwp=2147.78164, odp=0.0000323, acid=14.7111022, eutt=55.7722302, eutf=1.9709617, eutm=8.3834083, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Brine-based Lithium',
               citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  

Steel = Material(name ='steel', gwp= 2121.152427, odp=0.0000524, acid=7.071973, eutt=20.1478856, eutf=0.9654292, eutm=2.0671433, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Steel', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Cast_iron = Material(name ='cast iron', gwp= 1759.710573, odp=0.0000443, acid=6.0061499, eutt=18.2339132, eutf=0.66155, eutm=1.7330535, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Cast iron', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Concrete = Material(name ='concrete', gwp= 90.91, odp=0.00000040125, acid=0.246868, eutt=1.546130613, eutf=0.015425534, eutm=0.140321324, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Concrete', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Glass = Material(name = 'glass', gwp = 1118.5, odp=0.0000141, acid=8.8362743, eutt=22.0973602, eutf=0.1263297, eutm=1.8534905, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'glass',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Aluminium = Material(name = 'aluminium', gwp = 5632.53, odp=0.0000578, acid=31.2744756, eutt=65.5836037, eutf=1.8348121, eutm=6.2774955, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'aluminium',
                 citation = 'https://www.researchgate.net/figure/Overall-environmental-impact-per-kg-Pts-kg-and-GWP-kg-of-CO2-eq-kg-aluminium-of_fig2_295243557. & Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Silicon = Material(name = 'silicon', gwp = 50137.8, odp=0.0012152, acid=188.0829961, eutt=469.8237153, eutf=18.9483911, eutm=46.3346166, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'silicon',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Copper = Material(name = 'copper', gwp = 8660.4, odp=0.0003416, acid=516.3936129, eutt=409.7976693, eutf=46.0940178, eutm=29.6096174, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'copper',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')


Declare Processes

In [65]:

scales = TemporalScale(discretization_list=[1])

Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', cons_max=100, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', label='Power generated', demand = True, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0,1))#(min(demand['COAST']), 1))


LiI = Process(name='LiI', storage=Power, vopex=1302, capex=0, fopex=0,  prod_max=100,
            storage_loss=0.11,  label='Lithium-ion battery', basis='MW', store_max=100)

WF = Process(name='WF', conversion={Wind: -1, Power: 0.85}, vopex=990, capex=0, fopex=0,
            prod_max=100, label='Wind mill array', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
            , varying_bounds = (0,1))

PV = Process(name='PV', conversion={Solar: -1, Power: 0.75}, vopex=567, capex=0, fopex=0,
            prod_max=100, label='Solar PV', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
            , varying_bounds = (0,1))

region = Location(name='region', processes={
                LiI, WF, PV}, scales=scales, label='some region wo dispatchable')
scenario = Scenario(name='example', demand={region: {Power:50}}, network=region, scales=scales, label='example scenario')
matrix = formulate(scenario=scenario, model_class=ModelClass.MPLP)
    # print(matrix)
    # return matrix



The problem has the following variables:
Resource inventory level (Inv) x 1
Exact resource discharge (Sf) x 0
Uncertain resource discharge (S) x 1
Exact resource availability (Af) x 2
Uncertain resource availability (A) x 0
Exact resource price (Cf) x 2
Uncertain resource price (C) x 0
Exact process production (Pf) x 2
Uncertain process production (P) x 2
 For a total of 8 (5 fixed, and 3 uncertain)


In [66]:
solution= solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


## Changes to OG problem to add MOO (emission objectives)

In [67]:
matrix['F'] = numpy.array([numpy.concatenate((i, numpy.array([0]*6)), axis = 0) for i in matrix['F']])

In [68]:
matrix['CRa'] = numpy.concatenate((numpy.eye(9),-numpy.eye(9)))

In [69]:
matrix['CRb'] = numpy.concatenate((numpy.array([[1] for i in range(9)]), numpy.array([[0] for i in range(9)])))

In [70]:
matrix['c'] = numpy.array([[0] for i in range(8)])

In [71]:
material_cons_LiI = {LithiumCarbonateR: 20}
material_cons_WF = {Steel: 109.9, Cast_iron: 8.805, Concrete: 398.7}
material_cons_PV = {Glass: 70, Steel: 56, Concrete: 48, Aluminium: 19, Silicon: 7, Copper: 7}
material_cons_dict = {'PV': material_cons_PV, 'WF': material_cons_WF, 'LiI': material_cons_LiI}
material_cons_dict2 = {i: {m.name: n for m,n in j.items()} for i,j in material_cons_dict.items()}
objective_dict = {i: {m.name: {a: b*material_cons_dict2[i][m.name] for a,b in m.emission_potentials_dict.items()} for m,n in j.items()} for i, j in material_cons_dict.items()}
def calculate_total_potentials(data):
    totals = {}
    for process, materials in data.items():
        totals[process] = {}
        for material, impacts in materials.items():
            for impact, value in impacts.items():
                if impact not in totals[process]:
                    totals[process][impact] = 0
                totals[process][impact] += value
    return totals

# Calculate and print total potentials
objective_dict = calculate_total_potentials(objective_dict)
H1 = numpy.zeros((3,8))
LiI_obj = numpy.array([[objective_dict['LiI'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
LiI_d_obj = numpy.array([[0] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
PV_obj = numpy.array([[objective_dict['PV'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
WF_obj = numpy.array([[objective_dict['WF'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
H2 = numpy.concatenate((numpy.zeros((6,4)), numpy.concatenate((LiI_obj, LiI_d_obj, PV_obj, WF_obj), axis = 1)), axis = 1)
matrix['H'] = numpy.concatenate((H1, H2)).transpose()

In [72]:
solution_moo = solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


In [74]:
len(solution.critical_regions)

8

In [75]:
len(solution_moo.critical_regions)

4

\begin{align}
x(\theta) &= A\theta + b\\
\lambda(\theta) &= C\theta + d\\
\Theta &:= \{\forall \theta \in \mathbf{R}^m: E\theta \leq f\}
\end{align}